In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
from math import pi

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap, cumsum
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [9]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/Mensual.tps"
df = parser(path);df

,tipo_movim,fecha_comprob,tipo_comprob,prefijo,nro_compro,descripcion,nro_cuit,imp_exento,imp_gravado,imp_gravado_10,imp_gravado_27,importe_iva,imp_iva10,imp_iva27,imp_perc_iva,imp_per_ing_brutos,retenc_gananc,imp_varios,imp_total,cod_provincia,periodo_contable
0,1,2013-02-13,4,0,2,Marengo Alberto (Sueldo),11111111111,4170.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4170.50,13,2013-02-28
1,5,2014-01-07,NaN,0,2047,Marengo Alberto (Sueldo),11111111111,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6284.65,0,2014-01-07
2,5,2014-01-07,NaN,0,2048,Monzon Leandro (Sueldo),11111111111,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5994.20,0,2014-01-07
3,5,2014-02-14,NaN,0,2049,Marengo Alberto (Sueldo),11111111111,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4333.40,0,2014-02-14
4,5,2014-02-14,NaN,0,2050,Monzon Leandro (Sueldo),11111111111,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4136.80,0,2014-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10629,1,2011-10-12,1,1,19385,Daniel H. Ingaramo,20160600072,0.00,1160.00,0.00,0.00,243.60,0.00,0.00,0.00,0.00,0.00,0.00,1403.60,13,2011-11-30
10630,1,2011-11-14,1,1,19547,Daniel H. Ingaramo,20160600072,0.00,896.90,0.00,0.00,188.36,0.00,0.00,0.00,0.00,0.00,0.00,1085.26,13,2011-11-14
10631,1,2011-11-14,1,1,19555,Daniel H. Ingaramo,20160600072,0.00,713.85,0.00,0.00,149.91,0.00,0.00,0.00,0.00,0.00,0.00,863.76,13,2011-11-14
10632,1,2011-10-27,1,1,19481,Daniel H. Ingaramo,20160600072,0.00,455.70,0.00,0.00,95.70,0.00,0.00,0.00,0.00,0.00,0.00,551.40,13,2011-11-30


In [5]:
from app.models import Mensual

mensual = Mensual.objects.values(
    'fecha_comprob',
    'periodo_contable',
    'tipo_movim__descripcion',
    'tipo_comprob',
    'prefijo',
    'cod_proveedor',
    'cod_proveedor__razon_social',
    'imp_total',)\
.filter(imp_total__gt=0)\
.order_by('periodo_contable')

df = DataFrame(mensual)
df = df.rename({
    'fecha_comprob': 'Fecha Comprobante',
    'tipo_comprob': 'Tipo Comprobante',
    'periodo_contable': 'Periodo Contable',
    'tipo_movim__descripcion': 'Tipo Movimiento',
    'cod_proveedor__razon_social': 'Razón Social',
    'imp_total': 'Importe',
    'prefijo': 'Prefijo'
},
    axis='columns')
df['Periodo Contable'] = to_datetime(df['Periodo Contable'])
df['Fecha Comprobante'] = to_datetime(df['Fecha Comprobante'])
# Diferenciar ingresos de egresos
df['Tipo'] = 'Egresos'
df.loc[(df['Tipo Movimiento'].str.contains('Cobranza', case=False)) | (
        df['Tipo Movimiento'].str.contains('Nota de Crédito', case=False)), 'Tipo'] = 'Ingresos'

In [6]:
df

,Fecha Comprobante,Periodo Contable,Tipo Movimiento,Tipo Comprobante,Prefijo,cod_proveedor,Razón Social,Importe,Tipo
0,2002-05-20,2002-05-20,Orden de Pago,None,0,85.0,Fundacion Castellanos,1349.60,Egresos
1,2005-05-11,2005-05-31,Factura Cuenta Corriente,3,1,32.0,Maria Cristina Minotti,18913.00,Egresos
2,2011-10-13,2011-10-13,Factura Cuenta Corriente,1,1,34.0,Comuna De Colonia Aldao,47.24,Egresos
3,2011-10-20,2011-10-20,Orden de Pago,None,0,2.0,Sucesion Heck Eldo Olivio,6231.50,Egresos
4,2011-10-25,2011-10-25,Orden de Pago,None,0,7.0,Erio Gonzalo Rodriguez,350.00,Egresos
...,...,...,...,...,...,...,...,...,...
10604,2017-06-04,2017-06-30,Factura Cuenta Corriente,None,0,28.0,Sancor Seguros,32.00,Egresos
10605,2017-06-25,2017-06-30,Factura Cuenta Corriente,4,1,442.0,Mutual Union Sunchales,2251.38,Egresos
10606,2017-06-27,2017-06-30,Factura Cuenta Corriente,None,0,126.0,Mutual Regional Ramona,8950.11,Egresos
10607,2017-06-27,2017-06-30,Factura Cuenta Corriente,4,0,126.0,Mutual Regional Ramona,5448.19,Egresos


In [13]:
today = datetime(2014, 3, 1)
df_filter = df[(df['Periodo Contable'].dt.month == today.month) &
               (df['Periodo Contable'].dt.year == today.year)]

In [74]:
df[df.Tipo == 'Ingresos']

,Fecha Comprobante,Periodo Contable,Tipo Movimiento,Tipo Comprobante,Prefijo,Razón Social,Importe,Tipo
22,2011-08-15,2011-11-30,Nota de Crédito,1,30,Plastar San Luis S.A.,2530.11,Ingresos
64,2011-11-15,2011-11-30,Nota de Crédito,1,26,Pioneer Argentina S.R.L.,9344.91,Ingresos
87,2011-12-27,2011-12-31,Nota de Crédito,1,2,Comercial Eusebia S.A.,34945.50,Ingresos
267,2012-03-27,2012-03-31,Nota de Crédito,1,4,Melgratti Distribuciones Srl,438.49,Ingresos
363,2012-04-25,2012-04-30,Nota de Crédito,1,4,Melgratti Distribuciones Srl,438.50,Ingresos
884,2012-07-23,2012-07-31,Nota de Crédito,1,4,Jorge Daniel Sara,250.68,Ingresos
1572,2013-05-03,2013-05-03,Nota de Crédito,1,0,Melgratti Distribuciones Srl,93533.10,Ingresos
1869,2013-01-24,2013-06-24,Nota de Crédito,1,4,Jorge Daniel Sara,154.92,Ingresos
2076,2013-07-31,2013-07-31,Nota de Crédito,1,0,Melgratti Distribuciones Srl,5272.47,Ingresos
2242,2013-08-29,2013-08-29,Nota de Crédito,1,3,Alberto J Villa,138.15,Ingresos


In [76]:
df_ingresos = df_filter.loc[df_filter['Tipo'].str.contains('Ingresos', case=False)]
df_egresos = df_filter.loc[df_filter['Tipo'].str.contains('Egresos', case=False)]

In [77]:
df_ingresos

,Fecha Comprobante,Periodo Contable,Tipo Movimiento,Tipo Comprobante,Prefijo,Razón Social,Importe,Tipo
3508,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,22520.20,Ingresos
3509,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20475.50,Ingresos
3510,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,36074.90,Ingresos
3511,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20815.10,Ingresos
3512,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20649.50,Ingresos
3513,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20531.10,Ingresos
3514,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20838.90,Ingresos
3515,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,7074.14,Ingresos
3516,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,17792.00,Ingresos
3517,2014-03-31,2014-03-31,Nota de Crédito,None,100,Coop. Guillermo Lehmann,20396.80,Ingresos


In [83]:
gb_ingresos = DataFrame(df_ingresos.groupby(['Tipo Movimiento'],
                                         as_index=False).Importe.sum())
gb_ingresos.sort_values('Importe', inplace=True)
gb_ingresos

,Tipo Movimiento,Importe
0,Nota de Crédito,207299.53


In [85]:
# gb_egresos = DataFrame(df_egresos.groupby(['Fecha Comprobante', 'Razón Social',
#                                            'Tipo Movimiento'],
#                                          as_index=False).Importe.sum())
gb_egresos = DataFrame(df_egresos.groupby(['Tipo Movimiento']).Importe.sum())
gb_egresos.sort_values('Importe', inplace=True)

source = ColumnDataSource(gb_egresos)
plot = figure(
    x_range=Range1d(start=0,
                    end=max(source.data['Importe'], ),
                    bounds='auto'),
    y_range=FactorRange(*source.data['Tipo Movimiento']),
    title='Sub Diario IVA compras')
plot.hbar(source=source,
         y='Tipo Movimiento',
         right='Importe',
         height=0.8)
show(plot)

In [33]:
source = ColumnDataSource(gb_mov_impu)
source_ingresos = ColumnDataSource(gb_mov_impu[gb_mov_impu['Tipo'] == 'Ingresos'])
source_egresos = ColumnDataSource(gb_mov_impu[gb_mov_impu['Tipo'] == 'Egresos'])
tooltips = [("Razón Social", "@{Razón Social}"),
            ("Tipo Movimiento", "@{Tipo Movimiento}"),
            ("Tipo Comprobante", "@{Tipo Comprobante}"),
            ("Importe total", "@Importe{$0.00}")]
hovertool = HoverTool(names=['circle'], tooltips=tooltips)
plot = figure(
    tools=["tap,reset,save,crosshair", hovertool],
    toolbar_location='above',
    x_axis_type="datetime",
    y_range=Range1d(start=0,
                    end=max(source.data['Importe'], ),
                    bounds='auto'),
    title='Sub Diario IVA compras'
)
# Ingresos
plot.line(x='Fecha Comprobante', y='Importe',
          source=source_ingresos, name='line',
          line_color='green')
plot.circle(x='Fecha Comprobante', y='Importe',
            fill_color="white", size=8, name='circle',
            source=source_ingresos,
            line_color='green')
# Egresos
plot.line(x='Fecha Comprobante', y='Importe',
          source=source_egresos, name='line',
          line_color='red')
plot.circle(x='Fecha Comprobante', y='Importe',
            fill_color="white", size=8, name='circle',
            source=source_egresos,
            line_color='red')
plot.sizing_mode = 'scale_both'
plot.yaxis.formatter = NumeralTickFormatter(
    format="$0")
plot.yaxis[0].ticker.desired_num_ticks = 20

show(plot)

KeyError: 'Tipo'